<a href="https://colab.research.google.com/github/JCaballerot/Recommender_Systems/blob/main/K_Nearest_Neighbors_Recommender/Book_Crossing_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 align=center><font size = 5> K-Nearest Neighbors
 Recommender</font></h1>

---

<center>
  <img src="https://storage.googleapis.com/kaggle-datasets-images/1661575/2726067/684ac0c4c14cb46d1047ccb620b45cac/dataset-cover.jpg?t=2021-10-21-03-18-09" width="800" height="300">
</center>


## Objetivo de este Notebook

1. Cargar y preprocesar un Dataset.
2. Realizar un sistema de recomendación basado en KNN.
3. Comprobar el performance del sistema.

## Tabla de Contenidos

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Contexto</a>  
2. <a href="#item32">Descargar y preparar el Dataset</a>  
6. <a href="#item34">Entrenamiento del modelo</a>  
6. <a href="#item34">Validación del modelo</a>  

</font>
</div>

## 1. Contexto


El conjunto de datos "Book-Crossing" (también conocido como BX) es una colección de datos relacionados con libros y reseñas de libros. Este conjunto de datos se centra en la interacción de los usuarios con libros y sus calificaciones, y es ampliamente utilizado en aplicaciones de sistemas de recomendación.



<b>Descripción de datos</b>

---

El conjunto de datos Book-Crossing contiene información sobre:

* <b>Libros:</b> Información sobre los libros, incluyendo su título, autor y año de publicación.

* <b>Usuarios:</b> Perfiles de los usuarios que interactúan con los libros, incluyendo su ID y ubicación.

* <b>Calificaciones:</b> Calificaciones numéricas que los usuarios asignan a los libros que han leído.

El conjunto de datos puede ser utilizado para varios propósitos, como la construcción de sistemas de recomendación de libros, el análisis de patrones de lectura y preferencias de los usuarios, y la investigación en el campo de la minería de datos y la inteligencia artificial.

---



<strong>Puede consultar este [link](https://www.kaggle.com/datasets/syedjaferk/book-crossing-dataset) para leer más sobre la fuente de datos Book Crossing.</strong>


## 2. Descargar y preparar Dataset

In [ ]:
# Download Book-Crossing Dataset
!curl -o dataset.zip "http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip"
!unzip dataset.zip
!ls -la

In [6]:
# Principales librerías
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore") # Turn off warnings


In [ ]:
ratings = pd.read_csv("BX-Book-Ratings.csv", sep=";", encoding="ISO-8859-1")
books   = pd.read_csv("BX-Books.csv",        sep=";", encoding="ISO-8859-1", error_bad_lines=False)
users   = pd.read_csv("BX-Users.csv",        sep=";", encoding="ISO-8859-1")

In [ ]:
users.head()

In [ ]:
books.head()

In [ ]:
ratings.head()

In [ ]:
print("  Users: {} \n  Books: {}\n  Ratings: {}".format(len(users), len(books), len(ratings)))


In [9]:
users.columns = users.columns.str.lower().str.replace('-', '_')
books.columns = books.columns.str.lower().str.replace('-', '_')
ratings.columns = ratings.columns.str.lower().str.replace('-', '_')

### 2.1. Data de usuarios

In [ ]:
users.head()

In [ ]:
users["age"].describe()

In [10]:
# Ejemplo de remoción de outliers
IQR = np.nanpercentile(users['age'], 75) - np.nanpercentile(users['age'], 25)
lower_threshold = np.nanpercentile(users['age'], 50) - 1.5*IQR
upper_threshold = np.nanpercentile(users['age'], 50) + 1.5*IQR

users = users[(users['age'] > lower_threshold) & (users['age'] < upper_threshold)]

In [ ]:
# Establecer el estilo de Seaborn (opcional)
sns.set(style="whitegrid")

# Crear el gráfico de barras
plt.figure(figsize=(12, 5))  # Ajusta el tamaño de la figura si es necesario
ax = sns.countplot(data=users, x='age', color='lightblue')

# Personalizar el eje x
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')

# Ajustar el tamaño de fuente de las etiquetas del eje x
ax.tick_params(axis='x', labelsize=8)

# Agregar etiquetas y título
plt.xlabel('Edad', fontsize=12)
plt.ylabel('Cantidad', fontsize=12)
plt.title('Distribución de Edad de usuarios', fontsize=14)

# Mostrar el gráfico
plt.tight_layout()
plt.show()


### 2.2. Data de libros

In [12]:
#dropping the image columns
books.drop(columns=['image_url_s', 'image_url_m', 'image_url_l'], inplace=True) # drop image-url columns

In [ ]:
books.head()

In [ ]:
books[books.book_title == 'The Lovely Bones: A Novel']

In [13]:
#converting years of publication to integer
books.year_of_publication = pd.to_numeric(books.year_of_publication, errors='coerce')

In [14]:
#replacing all years of publication that are 0 with NaN
books.year_of_publication.replace(0, np.nan, inplace=True)

In [ ]:
books.year_of_publication.describe()

In [15]:
# Ejemplo de remoción de outliers
lower_threshold = 1964
upper_threshold = 2004

books = books[(books['year_of_publication'] >= lower_threshold) & (books['year_of_publication'] <= upper_threshold)]
books.year_of_publication = books.year_of_publication.astype(int)

In [ ]:
# Establecer el estilo de Seaborn (opcional)
sns.set(style="whitegrid")

# Crear el gráfico de barras
plt.figure(figsize=(12, 5))  # Ajusta el tamaño de la figura si es necesario
ax = sns.countplot(data=books, x='year_of_publication', color='lightblue')

# Personalizar el eje x
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')

# Ajustar el tamaño de fuente de las etiquetas del eje x
ax.tick_params(axis='x', labelsize=8)

# Agregar etiquetas y título
plt.xlabel('Año de Publicación', fontsize=12)
plt.ylabel('Cantidad', fontsize=12)
plt.title('Distribución de Años de Publicación de Libros', fontsize=14)

# Mostrar el gráfico
plt.tight_layout()
plt.show()


In [16]:
#correcting publisher names and assigning the name 'Other' to those with missing publisher names
books.publisher= books.publisher.str.replace('&amp;', '&', regex=False)

In [17]:
books.publisher.replace(np.nan,'Other', inplace = True)

In [18]:
#replacing the NaN in for book_author with Unknown
books.book_author.replace(np.nan,"Unknown", inplace=True)

In [19]:
#dropping the rows with NaN year of publication
books = books.dropna(how='any', axis = 0)

### 2.3. Data de Ratings

In [ ]:
ratings.head()

In [20]:
#removing the rows with an implicit book_rating of 0
ratings = ratings[ratings.book_rating!=0]

In [ ]:
ratings.book_rating.hist()

In [ ]:
# Establecer el estilo de Seaborn (opcional)
sns.set(style="whitegrid")

# Crear el gráfico de barras
plt.figure(figsize=(12, 5))  # Ajusta el tamaño de la figura si es necesario
ax = sns.countplot(data=ratings, x='book_rating', color='lightblue')

# Personalizar el eje x
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')

# Ajustar el tamaño de fuente de las etiquetas del eje x
ax.tick_params(axis='x', labelsize=8)

# Agregar etiquetas y título
plt.xlabel('Rating del libro', fontsize=12)
plt.ylabel('Cantidad', fontsize=12)
plt.title('Distribución de Rating de libros', fontsize=14)

# Mostrar el gráfico
plt.tight_layout()
plt.show()


### 2.4. Unificando data

In [ ]:
df_unified = pd.merge(users[['user_id', 'age']], ratings, on = 'user_id', how = 'inner')
df_unified = pd.merge(df_unified, books[['isbn', 'book_title']], on = 'isbn', how = 'inner')

df_unified.head()

In [ ]:
df_unified.loc[df_unified.user_id ==  387]

In [30]:
most_popular = df_unified.groupby('book_title')[['isbn']].count().reset_index()
most_popular.rename(columns = {'isbn' : 'popularity'}, inplace = True)
most_popular.sort_values(by = 'popularity', ascending = False, inplace = True)


In [ ]:
print(len(most_popular), 'diferentes ítems en el sistema')

In [63]:
#Nos quedaremos con los ítems con cierta materialidad de popularidad
print(len(most_popular[most_popular.popularity > 150]), 'diferentes ítems utilizados en el sistema')
items = most_popular[most_popular.popularity > 150].book_title.tolist()

32 diferentes ítems utilizados en el sistema


In [ ]:
df_unified_filtered = df_unified[df_unified.book_title.isin(items)]
df_unified_filtered.head()

# 3. K-Nearest Neighbors

K-Nearest Neighbors (KNN) es un algoritmo de machine learning que también se puede utilizar en sistemas de recomendación. La idea detrás del uso de KNN en sistemas de recomendación es encontrar usuarios o elementos similares en función de sus calificaciones o comportamientos previos y utilizar esa similitud para hacer recomendaciones.



* <b>Matriz de usuario-elemento:</b> Se crea una matriz que representa las calificaciones de los usuarios para los elementos. Cada fila de la matriz representa un usuario, y cada columna representa un elemento. Los valores de la matriz son las calificaciones dadas por los usuarios a los elementos.

* <b>Recomendación:</b>  Se utilizan las calificaciones de los K usuarios o elementos más cercanos para generar recomendaciones para el usuario o elemento en cuestión. Esto se puede hacer de varias maneras, como calcular un promedio ponderado de las calificaciones de los vecinos o identificar los elementos mejor calificados por los vecinos.

* <b> Evaluación:</b>  Se evalúa el rendimiento del sistema de recomendación utilizando métricas como RMSE (Root Mean Squared Error) o MAE (Mean Absolute Error) en un conjunto de datos de prueba para medir cuán precisas son las recomendaciones.

---


### 3.1. Muestreo de datos


El conjunto de datos en machine learning se divide típicamente en dos partes: el conjunto de entrenamiento (train) y el conjunto de prueba (test). Estas divisiones se utilizan para entrenar y evaluar los modelos.



<b>Train:</b> El conjunto de entrenamiento se utiliza para entrenar el modelo de machine learning. Es aquí donde el modelo "aprende" los patrones y relaciones en los datos para poder hacer predicciones o clasificaciones.

<b>Test:</b> El conjunto de prueba se utiliza para evaluar el rendimiento del modelo en datos no vistos durante el entrenamiento. Es una medida objetiva de la capacidad del modelo para generalizar y realizar predicciones precisas en nuevos datos.

In [211]:
# Muestreo
#La función train_test_split de scikit-learn se utiliza para dividir un conjunto de datos en subconjuntos de train y test.
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_unified_filtered, # Base de datos
                               train_size = 0.7, # Especificar el tamaño de train/test
                               random_state = 123) # Semilla aleatoria



In [212]:
# Crear una matriz pivot para el conjunto de entrenamiento
pivot_table_train = train.pivot(index='user_id', columns='isbn', values='book_rating').fillna(0)

# Crear una matriz pivot para el conjunto de prueba
pivot_table_test = test.pivot(index='user_id', columns='isbn', values='book_rating').fillna(0)


### 3.2. KNN recommender


In [171]:
from sklearn.neighbors import NearestNeighbors

In [244]:
# Crear el modelo k-NN
k = 50  # Número de vecinos más cercanos
model = NearestNeighbors(n_neighbors = k,
                                 metric='cosine',
                                 algorithm='brute',
                                 n_jobs=-1)

model.fit(pivot_table_train.values) # Ajustar el modelo a los datos




NearestNeighbors(algorithm='brute', metric='euclidean', n_jobs=-1,
                 n_neighbors=50)

In [ ]:

# Función para obtener recomendaciones para un usuario específico
def get_recommendations(user_ratings):
    distances, indices = model_knn.kneighbors([user_ratings], n_neighbors=k+1)  # +1 para excluir el propio usuario

    # Obtener los índices de los usuarios más cercanos (excluyendo el propio usuario)
    neighbor_indices = indices[0][1:]

    # Filtrar las calificaciones de los vecinos más cercanos
    neighbor_ratings = pivot_table_entrenamiento.iloc[neighbor_indices]

    # Calcular la puntuación promedio de los libros no calificados por el usuario
    book_scores = neighbor_ratings.mean()

    # Filtrar los libros que el usuario aún no ha calificado
    user_unrated_books = book_scores.index[~np.isnan(book_scores) & (user_ratings == 0)]

    # Ordenar los libros por puntuación promedio en orden descendente para obtener las recomendaciones
    recommendations = book_scores[user_unrated_books].sort_values(ascending=False)

    return recommendations


# Crear una tabla para almacenar las recomendaciones
recomendaciones_tabla = pd.DataFrame(columns=['user_id', 'isbn', 'puntuacion'])

# Para cada usuario en el conjunto de prueba, obtener sus recomendaciones
for user_id in pivot_table_prueba.index:
    if user_id in pivot_table_entrenamiento.index:  # Verificar si el usuario existe en el conjunto de entrenamiento
        user_ratings = pivot_table_prueba.loc[user_id].values
        recommendations = get_recommendations(user_ratings)
        # Agregar las recomendaciones a la tabla
        for isbn, score in recommendations.head(32).items():  # Tomar las 32 mejores recomendaciones
            recomendaciones_tabla = recomendaciones_tabla.append({'user_id': user_id, 'isbn': isbn, 'puntuacion': score}, ignore_index=True)




In [246]:
# Mostrar las primeras filas de la tabla de recomendaciones
recomendaciones_tabla.head()

,user_id,isbn,puntuacion
0,114,006017322X,0.0
1,114,044023722X,0.0
2,114,0670880728,0.0
3,114,0670032379,0.0
4,114,0618346252,0.0


In [247]:
recomendaciones_tabla[recomendaciones_tabla.puntuacion > 0].head()

,user_id,isbn,puntuacion
10,254,043935806X,3.04
11,254,059035342X,1.38
12,254,0345337662,0.44
13,254,0446310786,0.40
14,254,0316769487,0.36


In [248]:
len(recomendaciones_tabla[recomendaciones_tabla.puntuacion > 0])

10591

In [249]:
test[test.user_id == 254].head()

,user_id,age,isbn,book_rating,book_title
2404,254,24.0,0671021001,7,She's Come Undone (Oprah's Book Club)
1008,254,24.0,0142001740,9,The Secret Life of Bees


In [251]:
test.head()

,user_id,age,isbn,book_rating,book_title
16438,110912,36.0,0440226430,10,Summer Sisters
36318,265620,56.0,0440241073,8,The Summons
24034,138543,21.0,0446310786,10,To Kill a Mockingbird
5210,185111,20.0,0316769487,8,The Catcher in the Rye
11890,77516,35.0,0743418174,9,Good in Bed


In [250]:
recsys = pd.merge(recomendaciones_tabla, test, on = ['user_id', 'isbn'], how = 'left')
recsys[~recsys.book_rating.isnull()].head()

,user_id,isbn,puntuacion,age,book_rating,book_title


In [228]:
test[test.user_id == 6251]

,user_id,age,isbn,book_rating,book_title
42480,6251,32.0,0670894605,8,The Secret Life of Bees
1696,6251,32.0,0439136350,10,Harry Potter and the Prisoner of Azkaban (Book 3)


---
## Gracias por completar este laboratorio!